In [ ]:
import numpy as np

from notebook_helpers import *

In [ ]:
from scipy.interpolate import (
    RBFInterpolator,
    RegularGridInterpolator,
    griddata,
    CloughTocher2DInterpolator,
    LinearNDInterpolator,
)
from scipy import ndimage

In [ ]:
alignment_volume_path = Path(
    "/home/ediun/.local/share/histalign/alignment_volumes/d2783eb27223868f57e159f01ce7a1b2.npz"
)

In [ ]:
alignment_volume = np.load(alignment_volume_path)["array"][::5, ::5, :256]
alignment_volume = convert_to_dtype(alignment_volume, np.uint8)
print_array_size(alignment_volume)
alignment_volume.shape

In [ ]:
vedo.Volume(alignment_volume).show()

In [ ]:
x_old = np.linspace(0, alignment_volume.shape[0] - 1, alignment_volume.shape[0])
y_old = np.linspace(0, alignment_volume.shape[1] - 1, alignment_volume.shape[1])
z_old = np.linspace(0, alignment_volume.shape[2] - 1, alignment_volume.shape[2])

In [ ]:
interpolator = RegularGridInterpolator(
    (x_old, y_old, z_old), alignment_volume, method="linear"
)

In [ ]:
x_new = np.linspace(
    0, alignment_volume.shape[0] // 2 - 1, alignment_volume.shape[0] // 2
)
y_new = np.linspace(
    0, alignment_volume.shape[1] // 2 - 1, alignment_volume.shape[1] // 2
)
z_new = np.linspace(
    0, alignment_volume.shape[2] // 2 - 1, alignment_volume.shape[2] // 2
)

In [ ]:
points = np.indices((len(x_new), len(y_new), len(z_new)), dtype=np.uint8)
points = points.reshape(len(x_new) * len(y_new) * len(z_new), 3)
print_array_size(points)

In [ ]:
points.shape

In [ ]:
interpolated_data = interpolator(points)
interpolated_data = interpolated_data.reshape(len(x_new), len(y_new), len(z_new))

In [ ]:
vedo.Volume(interpolated_data).show()

# Testing

In [ ]:
target_coordinates = np.array(np.nonzero(alignment_volume == 0))

In [ ]:
interpolated_data = ndimage.map_coordinates(
    alignment_volume, target_coordinates
).astype(alignment_volume.dtype)

In [ ]:
interpolated_volume = np.zeros_like(alignment_volume)

In [ ]:
for i in range(target_coordinates.shape[1]):
    interpolated_volume[target_coordinates[:, i]] = interpolated_data[i]

In [ ]:
vedo.Volume(interpolated_data)

In [ ]:
target_coordinates = np.array(np.nonzero(alignment_volume == 0))
interpolated_data = ndimage.map_coordinates(alignment_volume, target_coordinates)

In [ ]:
interpolated_data.max()

In [ ]:
interpolated_volume = np.zeros_like(alignment_volume)

fancy_indices = tuple(map(tuple, target_coordinates.tolist()))
interpolated_volume[fancy_indices] = interpolated_data

In [ ]:
filtered_volume = ndimage.spline_filter(alignment_volume, order=2)

In [ ]:
volume = vedo.Volume(filtered_volume)
volume.cmap(**get_cmap(volume))
volume.show()

In [ ]:
value_coordinates = np.array(np.nonzero(alignment_volume > 0), dtype=np.uint16).T
value_coordinates.shape

In [ ]:
target_coordinates = np.array(np.nonzero(alignment_volume == 0), dtype=np.uint16).T
target_coordinates.shape

In [ ]:
values = alignment_volume[np.nonzero(alignment_volume)]
values.shape

In [ ]:
y = value_coordinates
d = values

interpolator = RBFInterpolator(y, d, kernel="linear")
interpolator

In [ ]:
print_size_from_bytes(12879873033316)

In [ ]:
value_coordinates = np.array(np.nonzero(alignment_volume > 0), dtype=np.uint16).T
values = alignment_volume[np.nonzero(alignment_volume)]

In [ ]:
target_coordinates = np.array(np.nonzero(alignment_volume == 0), dtype=np.uint16).T

In [ ]:
linear_interpolated = griddata(
    value_coordinates, values, target_coordinates, method="linear"
)

In [ ]:
known_values_coordinates = np.nonzero(alignment_volume)
known_values = alignment_volume[known_values_coordinates]

In [ ]:
points = np.array(known_values_coordinates).T
values = known_values

interpolator = LinearNDInterpolator(points, values)

In [ ]:
target_values_coordinates = np.nonzero(alignment_volume == 0)
target_values = interpolator(target_values_coordinates)

In [ ]:
target_values.max()